# Probabilistic Forecasting with `sktime`: Distribution Forecast

## Notebook Goal: 
* Provide an example workflow of a distribution forecast with BGL data using sktime.
* Demonstrate how to use Optuna for a single forecasting type.

In [1]:
%load_ext autoreload
%autoreload 2

## Data Load

In [3]:
from src.data.data_loader import load_data

# Specify the columns to keep
keep_columns =['id', 'p_num', 'time', 'bg-0:00', 'insulin-0:00', 'carbs-0:00', 'hr-0:00', 'steps-0:00', 'cals-0:00', 'activity-0:00']

# Load the data with specified columns
df = load_data(data_source_name='kaggle_brisT1D', dataset_type='train', keep_columns=keep_columns)

# Print the first 5 rows of the data
print(df.head())

      id p_num      time  bg-0:00  insulin-0:00  carbs-0:00  hr-0:00  \
0  p01_0   p01  06:10:00     15.1        0.0417         NaN      NaN   
1  p01_1   p01  06:25:00     14.4        0.0417         NaN      NaN   
2  p01_2   p01  06:40:00     13.9        0.0417         NaN      NaN   
3  p01_3   p01  06:55:00     13.8        0.0417         NaN      NaN   
4  p01_4   p01  07:10:00     13.4        0.0417         NaN      NaN   

   steps-0:00  cals-0:00 activity-0:00  
0         NaN        NaN           NaN  
1         NaN        NaN           NaN  
2         NaN        NaN           NaN  
3         NaN        NaN           NaN  
4         NaN        NaN           NaN  


/home/cjrisi/Projects/diabetes/nocturnal-hypo-gly-prob-forecast/src/data/data_loader.py:26: DtypeWarning: Columns (506) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path, usecols=keep_columns)


In [4]:
from sktime.forecasting.arima import ARIMA
from sktime.forecasting import ForecastingOptunaSearchCV

# step 1: data specification
y = df['bg-0:00']
# step 2: specifying forecasting horizon
fh = [1, 2, 3]
# step 3: specifying the forecasting algorithm
forecaster = ARIMA()
# step 4: fitting the forecaster
forecaster.fit(y, fh=[1, 2, 3])
# step 5: querying predictions
y_pred = forecaster.predict()

# for probabilistic forecasting:
#   call a probabilistic forecasting method after or instead of step 5
y_pred_int = forecaster.predict_interval(coverage=0.9)
y_pred_int

/home/cjrisi/Projects/diabetes/nocturnal-hypo-gly-prob-forecast/.noctprob-venv/lib/python3.12/site-packages/gluonts/json.py:102: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


bg-0:00           
              0.9           
            lower      upper
177024  10.782319  12.334859
177025  10.426719  12.608672
177026  10.149430  12.805196

In [ ]:
from sktime.forecasting.model_selection import (
    ForecastingOptunaSearchCV,
    )
from sktime.datasets import load_shampoo_sales
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
from sktime.forecasting.base import ForecastingHorizon
from sktime.split import ExpandingWindowSplitter
from sktime.split import temporal_train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sktime.forecasting.compose import TransformedTargetForecaster
from sktime.transformations.series.adapt import TabularToSeriesAdaptor
from sktime.transformations.series.detrend import Deseasonalizer, Detrender
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.theta import ThetaForecaster
from sktime.forecasting.trend import STLForecaster
import optuna
from  optuna.distributions import CategoricalDistribution